In [9]:
import pandas as pd
import numpy as np

# Carregar Dados

In [3]:
df_rupturas = pd.read_csv('../data/dados_ruptura.csv')
df_rupturas['is_ruptura'] = df_rupturas['is_ruptura'].apply(int)
df_rupturas.head()

,key_loja,cod_produto,dt_estoque,primeira_data,qt_estoque,is_ruptura
0,1,1428,2024-01-01,2024-01-01,108.0,0
1,1,1428,2024-01-02,2024-01-01,108.0,0
2,1,1428,2024-01-03,2024-01-01,108.0,0
3,1,1428,2024-01-04,2024-01-01,108.0,0
4,1,1428,2024-01-05,2024-01-01,108.0,0


# 1. Os similares estão ajudando a evitar perda de vendas em situações de ruptura?

## 1.1. Quantas rupturas ocorreram? (SKU, loja, data)

In [10]:
total_rupturas = df_rupturas['is_ruptura'].sum()
proporcao = np.round((total_rupturas/df_rupturas.shape[0])*100, 2)
print(f"Total de eventos de ruptura: {total_rupturas} ({proporcao}% dos registros de estoque)")

Total de eventos de ruptura: 113186 (40.04% dos registros de estoque)


### Histórico Rupturas

In [12]:
import matplotlib.pyplot as plt

rupturas_por_dia = df_rupturas[df_rupturas['is_ruptura']].groupby('dt_estoque').size()

plt.figure(figsize=(12,6))
rupturas_por_dia.plot()
plt.title('Evolução das Rupturas ao Longo do Tempo')
plt.xlabel('Data')
plt.ylabel('Quantidade de Rupturas')
plt.grid()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

### Loja

### SKU

In [ ]:
## 1.2. Quantas dessas rupturas tinham similares recomendados disponíveis em estoque? ->

In [ ]:
## 1.3. Quantas dessas situações resultaram em vendas de similares? ->

In [ ]:
## 1.4. Qual a taxa de substituição geral? (substituição bem-sucedida ÷ total de rupturas) ->

In [ ]:
## 1.5. Qual o tempo médio entre a ruptura e a venda do similar? ->

In [ ]:
# Em que contextos os similares são mais aceitos?

In [ ]:
## 2.1. A substituição é maior para quais rankings de similaridade? ->

In [ ]:
## 2.2. O canal de venda influencia na substituição? ->

In [ ]:
## 2.3. Há variação por categoria, subcategoria ou linha do produto? ->

In [ ]:
## 2.4. O preço do similar afeta a substituição? (mais barato = mais aceito?) ->

In [ ]:
## 2.5. Há diferenças por região (UF)? ->

In [ ]:
## 2.6. A substituição é mais rápida em certos contextos? ->

In [ ]:
# O que a empresa pode fazer com essas informações?

In [ ]:
## 3.1. Quais tipos de SKU se beneficiam mais de ter similares em estoque? ->

In [ ]:
## 3.2. Onde (lojas, categorias, canais) o mix de similares deve ser revisto? ->

In [ ]:
## 3.3. O algoritmo de recomendação precisa de ajustes? (ex: ranking 1 não converte bem?) ->

In [ ]:
## 3.4. Como priorizar o reabastecimento de similares com maior taxa de conversão? ->